In [1]:
import argparse
import os,cv2
import random,numpy,pandas
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [2]:
import argparse
import os
import random,numpy
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

Random Seed:  999


In [4]:
fsc_test={}
for i in os.listdir('/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test'):
    fsc_test[i]=TF.to_tensor(cv2.resize(cv2.imread(f'/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/{i}'),(256, 256)))

print(fsc_test["21.jpg"].shape)
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

torch.Size([3, 256, 256])


In [5]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class FSC(nn.Module):
    def __init__(self):
        super().__init__()
        self.FSC_rafire=nn.Sequential(
            nn.Conv2d(3, 16, 3),  # Increased channels
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(16, 64, 3),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 126, 3),
            nn.BatchNorm2d(126),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(126, 256, 3),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(256, 512, 3),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(512, 1024, 3),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Flatten(),
            
            nn.Linear(4096, 2)  
        )

    def forward(self,x):
        rsc_rafire = self.FSC_rafire(x)
        return  torch.argmax(rsc_rafire, dim=1)

In [6]:
FSC_net = FSC().to(device)

FSC_net.load_state_dict(torch.load("/kaggle/input/d/mafiosoquasar/fake-scene-classification-model-2/0.014429506845772266 97.14286041259766 77.pth",weights_only=False,map_location=torch.device('cpu')))


<All keys matched successfully>

In [7]:
fsc_submission=pandas.read_csv("/kaggle/input/cidaut-ai-fake-scene-classification-2024/sample_submission.csv", index_col ="image")#.to_dict('list')

z_add,z_total=0,0
for i in fsc_submission.index:
    img=fsc_test[i].reshape((1, 3, 256, 256)).float().to(device)
    #print(FSC_net(img))
    
    output = FSC_net(img).cpu().detach().numpy()
    if output==0:
        z_add+=1
    z_total+=1
    fsc_submission.loc[i]['label']=output

print(z_total,z_add)
fsc_submission=pandas.DataFrame(fsc_submission)
fsc_submission['image']=fsc_submission.index
fsc_submission.to_csv(f"submission.csv", index=False)
fsc_submission

/tmp/ipykernel_17/774619272.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  fsc_submission.loc[i]['label']=output


180 77


,label,image
image,,
102.jpg,1,102.jpg
108.jpg,1,108.jpg
109.jpg,0,109.jpg
111.jpg,1,111.jpg
121.jpg,1,121.jpg
...,...,...
899.jpg,1,899.jpg
91.jpg,1,91.jpg
94.jpg,0,94.jpg
